<a href="https://colab.research.google.com/github/kmaciver/newsflash/blob/master/modelTraining/SimpleTextGenerator_ver0(Bidirectional_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Introduction

The idea of this notebook is to model a simple text generator to predict next word on business headlines

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import os
import pickle
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import warnings
warnings.filterwarnings('ignore')

## Data preparation 

In [0]:
Data_train = pd.read_csv('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Dataset/train.csv', names=['Category Id','Title','Description'], header=0)
Data_train.head()

,Category Id,Title,Description
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...


In [0]:
Data_test = pd.read_csv('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Dataset/test.csv', names=['Category Id','Title','Description'], header=0)
Data_test.head(10)

,Category Id,Title,Description
0,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
1,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
2,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
3,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
4,4,Open Letter Against British Copyright Indoctri...,The British Department for Education and Skill...
5,4,Loosing the War on Terrorism,"\\""Sven Jaschan, self-confessed author of the ..."
6,4,"FOAFKey: FOAF, PGP, Key Distribution, and Bloo...",\\FOAF/LOAF and bloom filters have a lot of i...
7,4,E-mail scam targets police chief,"Wiltshire Police warns about ""phishing"" after ..."
8,4,"Card fraud unit nets 36,000 cards","In its first two years, the UK's dedicated car..."
9,4,Group to Propose New High-Speed Wireless Format,LOS ANGELES (Reuters) - A group of technology...


In [0]:
data = pd.concat([Data_train, Data_test], ignore_index=True)
data.head()

,Category Id,Title,Description
0,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...


In [0]:
data= data.loc[data['Category Id']==3]

In [0]:
data = data.drop(columns=['Category Id'])

Looking at the text is possible to see that some words aren't going to be usefull for our purpose.

Examples are:
* Numbers
* Html references
* Names of the publishers that appear twice, e.g. (Reuters) Reuters, (AP) AP etc 

In [0]:
def publishers(text):
    pub = re.search(r"\(.*\)",text)
    if pub != None:
        return(pub.group(0))
    else:
        return("")

data['publishers']=data['Title'].apply(publishers)
data.head()

,Title,Description,publishers
0,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,(Reuters)
1,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,(Reuters)
2,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,(Reuters)
3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",(AFP)
4,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...,(Reuters)


In [0]:
def htmlLinks(text):
    links = re.search(r"&lt;A HREF.+?(target.+;.+;)",text)
    if links != None:
        return(links.group(0))
    else:
        return("")

data['links']=data['Description'].apply(htmlLinks)

In [0]:
def clean_publishers(text,publisher):
    if publisher != "":
        return(text.replace(publisher," "))
    else:
        return(text)
    
data['Title'] = np.vectorize(clean_publishers)(data['Title'], data['publishers'])

def clean_publishers_description(text,publisher):
    if publisher != "":
        return(text.replace(publisher[1:-1]," "))
    else:
        return(text)

data['Description']= np.vectorize(clean_publishers_description)(data['Description'], data['publishers'])

In [0]:
def clean_links(text,links):
    if links != "":
        return(text.replace(links," "))
    else:
        return(text)
    
data['Description'] = np.vectorize(clean_publishers)(data['Description'], data['links'])

In [0]:
len(data)

31898

In [0]:
data['length_Title'] = data['Title'].str.len() 
data['length_Description'] = data['Description'].str.len()

In [0]:
np.savetxt(r'/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Dataset/data.txt', data.iloc[:,0:2].values, fmt='%s')

In [0]:
text = open('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Dataset/data.txt', 'rb').read().decode(encoding='utf-8')

## Generating vocabulary on most frequent words

In [0]:
tokens = text_to_word_sequence(text,  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r0123456789'+"'")


In [0]:
len(tokens)

1186071

In [0]:
vocab_dict={}

for word in tokens:
    if word in vocab_dict:
        vocab_dict[word] += 1
    else:
        vocab_dict[word] = 1

vocab_words = []
vocab_freq=[]

for k,v in vocab_dict.items():
    vocab_freq.append(v)
    vocab_words.append(k)

In [0]:
wordFreqDf = pd.DataFrame(list(zip(vocab_words, vocab_freq)), columns =['Words', 'Freq'])
wordFreqDf = wordFreqDf.sort_values(by='Freq', ascending=False)

# Get words the top 20% frequently used words
TopwordFreqDf = wordFreqDf[wordFreqDf['Freq']>=np.percentile(wordFreqDf['Freq'].values,80)]
vocab = TopwordFreqDf['Words'].values

Since the model will be trained on the most common words, a Wildcard Token will be added to the vocab in order to deal with words outside of the top words vocabulary

In [0]:
vocab = np.append(vocab, '<Unkown>')
vocab = list(vocab)
print ('{} unique characters'.format(len(vocab)))

5567 unique characters


## Generating emmbeding matrix based on Glove 

In [0]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    model = {}
    with open(gloveFile,encoding="utf8") as f:
        for line in f:
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [0]:
gloveEmb = loadGloveModel('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Dataset/Glove Embedding/glove.6B.100d.txt')

Loading Glove Model
Done. 400000  words loaded!


Because of the data and the techniques applied during cleaning, it is possible to have some **Out-Of-Vocabulary words (OOV)**, that means words that are not in the downloaded Glove dictionary

In [0]:
embedded = np.zeros(shape=(len(vocab),100))

for i in range(0,len(vocab)):
    try:
        embedded[i] = gloveEmb[vocab[i]]
    except:
        continue

In [0]:
embedded.shape

(5567, 100)

In [0]:
import pickle

# Save vocab
with open('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/DocsToLoad/vocab.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(vocab, filehandle)

# Save embedded
np.save('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/DocsToLoad/embedded.npy', embedded)

## Create Train data

In [0]:
# Separate text file by lines
TextLines = open('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Dataset/data.txt').readlines()

In [0]:
len(TextLines)

31898

In [0]:
from sklearn.model_selection import train_test_split

# Validation list
TrainLines, TestLines = train_test_split(TextLines, test_size=0.10, random_state=42)

print(len(TrainLines),len(TestLines))

28708 3190


In [0]:
def generate_text_sequences(Lines, pastWords, vocab):
    X_line = list()
    Y_line = list()
    pastWords = pastWords
    for line in Lines:
        # Tokenize line
        lineTokenized = text_to_word_sequence(line,  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r0123456789'+"'")
        #Get line length
        lengthLine = len(lineTokenized)
        lineBatch = lengthLine-pastWords
        
        # Substitute words outside vocab with <Unkown>
        for idx in range(0,len(lineTokenized)):
            if lineTokenized[idx] in vocab:
                continue
            else:
                lineTokenized[idx] = '<Unkown>'
        
        #Crate sequences of text 
        for i in range(0,lineBatch):
            X_sequence = lineTokenized[i:i+pastWords]
            X_line.append(X_sequence)
            Y_sequence = lineTokenized[i+pastWords]
            Y_line.append(Y_sequence)
    
    return(X_line, Y_line)

In [0]:
pastWords = 5  # number of words to look back for prediction
X_lineTrain, Y_lineTrain = generate_text_sequences(TrainLines, pastWords, vocab)

NameError: ignored

In [0]:
print(len(X_lineTrain), len(Y_lineTrain))

NameError: ignored

Creating a batch generator for training data. Converting the whole dataset will take too much memory

In [0]:
from tensorflow.keras.utils import to_categorical
import random as rand
from random import randint
from random import seed
rand.seed(4)

def batch_generator_data(batchsize, X_line, Y_line, embDim, pastWords, embedded, vocab):
    embDim=embDim
    pastWords = pastWords
    x_batch = np.zeros(shape=(batchsize,pastWords,embDim))
    y_batch = np.zeros(shape=(batchsize))

    while True:
        # Fill the batch with random continuous sequences of data.

        # Get a random start-index.
        # This points somewhere into the data.
        idx = np.random.randint(len(X_line) - batchsize)

        for i in range(0,batchsize):
            x_batch[i] = [embedded[vocab.index(x)] for x in X_line[idx+i]]
            y_batch[i] = vocab.index(Y_line[idx+i])

        #y_batch = to_categorical(y_batch, num_classes=len(vocab))
        
        yield (x_batch, y_batch)

In [0]:
embDim = 100 #shape of the embbeded latent space
batchsize = 300 #batch size for each training step
generator = batch_generator_data(batchsize,X_lineTrain, Y_lineTrain, embDim, pastWords, embedded, vocab)

NameError: ignored

In [0]:
X_train_batch, Y_train_batch = next(generator)

In [0]:
print(X_train_batch.shape)
print(Y_train_batch.shape)

(300, 5, 100)
(300,)


**Generate Validation data**

In [0]:
X_lineTest, Y_lineTest = generate_text_sequences(TestLines, pastWords, vocab)
valgenerator = batch_generator_data(batchsize,X_lineTest, Y_lineTest, embDim, pastWords, embedded, vocab)

In [0]:
print(len(X_lineTest), len(Y_lineTest))

103138 103138


In [0]:
X_test_batch, Y_test_batch = next(valgenerator)

In [0]:
print(X_test_batch.shape)
print(Y_test_batch.shape)

(300, 5, 100)
(300,)


## Create Model 

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, LSTM, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from keras import losses


model = Sequential()
model.add(Bidirectional(LSTM(units=500, return_sequences=True),input_shape=(pastWords,embDim)))
model.add(LSTM(units=200))
model.add(Dense(len(vocab), activation='softmax'))
model.summary()

Using TensorFlow backend.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 5, 1000)           2404000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               960800    
_________________________________________________________________
dense (Dense)                (None, 5567)              1118967   
Total params: 4,483,767
Trainable params: 4,483,767
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

optimizer = RMSprop(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

model_file = "/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/train_log/simpleTextGenerator.{epoch:02d}-{loss:.2f}.h5"

mc = ModelCheckpoint(model_file, monitor="loss", mode="min", save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=1e-4)

es = EarlyStopping(monitor='loss', min_delta=0.01, patience=5, mode='min')

In [0]:
%%time
#model = tf.keras.models.load_model('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/simpleTextGenerator.46-3.28.h5')

history = model.fit_generator(generator=generator,
                    epochs=500,
                    steps_per_epoch= len(X_lineTrain)//batchsize,
                    validation_data=valgenerator,
                    validation_steps= len(X_lineTest)//batchsize,
                    callbacks=[mc, reduce_lr, es])
                    #,initial_epoch=46)

In [0]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = model_file.split('.')[0]+'.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [0]:
saveDrivePath = '/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/'
h5FilePath = '/content/'+ model_file
#historyFilePath = '/content/' + hist_csv_file

os.system("mv "+'"'+h5FilePath+'"' + " " + '"'+saveDrivePath+'"' )
os.system("mv "+'"'+historyFilePath+'"' + " " + '"'+saveDrivePath+'"' )

0

## Load Model, Vocab and Embedded if already trained 

In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/DocsToLoad/simpleTextGenerator.56-3.08.h5')

# Read Vocabulary of data
vocab = []

with open('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/DocsToLoad/vocab.data', 'rb') as filehandle:
    # read the data as binary data stream
    vocab = pickle.load(filehandle)
    
print ('{} unique characters'.format(len(vocab)))

5567 unique characters


In [0]:
# Read embedded spaces of vocab data

embedded = np.load('/content/drive/My Drive/SharpestMinds/3. Generate Language Model/Bidirectional_LSTM_model/DocsToLoad/embedded.npy')

print(embedded.shape)

(5567, 100)


## Evaluate Model

For this project we'll like to get the top 4 predictions for the next word. The model will be evaluated by checking if the next word was in the top 4 predictions made by the model. Then we'll calculate the rate of sucess were that ocurred.



In [0]:
def generateInputArray(inputs):
  embDim = embedded.shape[1]
  x_sample =[]
  for x in inputs:
    if x in vocab:
      x_sample.append(embedded[list(vocab).index(x)])
    else:
      x_sample.append(np.zeros(embDim))

  x_sample = np.array(x_sample)
  x_sample = np.expand_dims(x_sample, axis=0)
  return(x_sample)
  
def generatecandidates(text):
  textToken = tf.keras.preprocessing.text.text_to_word_sequence(str(text),  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r0123456789'+"'")   
  
  x_sample = generateInputArray(textToken[-5:])
  
  y_sample = model.predict(x_sample)
  
  # Get top 50 candidates
  ind = np.argpartition(y_sample[0,:], -50)[-50:]

  candidates=dict()
  for i in ind:
    if vocab[i] != "<Unkown>":
      candidates[vocab[i]] = y_sample[0,i]

  return(candidates)

In [0]:
X_line, Y_line = generate_text_sequences(TestLines, pastWords, vocab)

In [0]:
points=0
TotalPredictions = 0
for i in range(0, len(TestLines)):
    # Verify if the correct word is in the top-50 words predicted
    candidates = generatecandidates(X_line[i])
    if Y_line[i] == '<Unkown>':
        continue
    else:
        if Y_line[i] in candidates.keys():
            word_prob = list(candidates.keys()).index(Y_line[i])
            p = np.array(list(candidates.values()))
            points += 1 - p[word_prob] 
            TotalPredictions +=1
        else:
            points+=1
            TotalPredictions +=1

print("Metric between the distribution probability of the next word", points)

Metric between the distribution probability of the next word 2433.4333511351497
